# Reduced Structure Function

**Author**: Mike & Jinafu

**Changes**
1. Update Condition for Monte Carlo randomizer
2. Update Condition to detect Magnetic Field

**Description**
A Monte Carlo way to calculate the structure Function.



## **Motivation**

Calculating Structure Function is always a headache.

Typically, to calculate the structure function of a particular data, we will have an extremely high complexity.

For example, in a 2D map, we need to select every point, and pair it with every other point in the plain. 

The complexity is $O(\frac{N^2(N^2+1)}{2}) \approx O(N^4)$, where $N$ is resolution in length. Say N = 792 in a 792x729 map, we will need at least 10min to calculate the function on a laptop.

In 3D map it's even more dreadful: for each cube you will need ~ $O(N^6)$ to calculate the simple structure function.

## **So, What can we do?**

How about we **randomly select** some **seed-points $P=\{p_1, p_2, ..., p_n\}$** in the cube, and only calculate **for a certain radius $R = [r_\min, r_\max]$**, so that we can restrict the complexity and also get the approximate result we want?

## **Concern**

Q1. What if it's not accurate?

A1. **Observe the result for a long time, and see if the result changes significantly.** No Monte Carlo algorithm is accurate. But think when you have a series of data very approximate with each other, then you can actually have an accurate upper bound and lower bound of the fluctuation of your result. What we're trying to do is to find good parameters so taht the radius and number of random seed points are chosen so that the result is note fluctuating. According to @Jianfu, when you choose `nRand = 50` and `R = 100`, the result is not significanlty change when you select different points. Since in a _stochastics environment_, when you have a data with its result fluctuate, you should be able to observe _a very big fluctuation_ when you do multiple experiments (say 100 experiments). **When you can't see this phenomenon under a certain set of parameters for a long time, you can assert that there is high chance that the structure function is stable, and the result we get is a good approximation to the original one.**



## **The Idea of the Algorithm**

1. Select a set of seed points $P=\{p_1, p_2, ..., p_n\}$
2. Select a range of radius $R = [r_\min, r_\max]$
3. For each point $p$ in the seed point set $P$, we do the following:
    1. Pick every point $q = p + \vec{r}$ on the plane/cube (so every $q$ point on the circle / the sphere) 
    2. Calculate `struct_function(p, q)` and store them
4. Calculate the average value for each radius $R$. We get the result

# The Code

In [2]:
using PyCall, PyPlot

In [3]:
# @param
#      A: 3dcube,  nx x ny x nz
#  nRand: Int pick nRand points
# @return
#    SF: Vector, 1 x R


function rand_out_struct3d(BX, BY, BZ, nRand, maxR)
    @time begin
        nx, ny, nz = size(BX);
        SF  = zeros(3*maxR^2 + 1);
        NF  = zeros(3*maxR^2 + 1);
        # @@ Optimization? Can you find a integer boundary

        # 1. Randomly pick points in A
        # rx, ry, rz: random points in the cube
        record = zeros(nx,ny,nz);
        rx = zeros(Int64, nRand , 1)
        ry = zeros(Int64, nRand , 1)
        rz = zeros(Int64, nRand , 1)
    end
    
    cnt = 1;
    maxIteration = 3 * nRand;
    @time while maxIteration > 0 && (cnt) <= nRand
       maxIteration -= 1;
       x = rand(1:nx)
       y = rand(1:ny)
       z = rand(1:nz)
       if record[x,y,z] == 0
          record[x,y,z] = 1
           if -1 < BX[x,y,z]<1 || -1 < BY[x,y,z] < 1 || -1 < BZ[x,y,z] < 1
               continue
           end
           rx[cnt] = x
           ry[cnt] = y
           rz[cnt] = z
           cnt += 1
        end
    end
    cnt -= 1
    nRand = min(nRand, cnt);

    # 2. For each point, calculate the cube
    # [x-R: x+R, y-R:y+R, z-R:z+R] (have to modular)
    @time for dx = -maxR:maxR, dy = -maxR:maxR, dz = -maxR:maxR

        R2_pq = round(Int64, dx^2 + dy^2 + dz^2);
        R = sqrt(R2_pq)
        
        
        for i in 1:nRand
            px, py, pz = rx[i], ry[i], rz[i];
#             println("$px, $py, $pz")
            qx = mod(px+dx, nx) + 1;
            qy = mod(py+dy, ny) + 1;
            qz = mod(pz+dz, nz) + 1;
            
            if -1 < BX[qx,qy,qz]<1 || -1 < BY[qx,qy,qz] < 1 || -1 < BZ[qx,qy,qz] < 1
                continue
            end
            dbx = BX[px,py,pz] - BX[qx,qy,qz];
            dby = BY[px,py,pz] - BY[qx,qy,qz];
            dbz = BZ[px,py,pz] - BZ[qx,qy,qz];
                        
            dbx = dbx * abs.(dx) / R
            dby = dby * abs.(dy) / R
            dbz = dbz * abs.(dz) / R
            
            SF[R2_pq+1] += dbx^2 + dby^2 + dbz^2;
            NF[R2_pq+1] += 1;
        
        end
    end
    
    return SF, NF
end




rand_out_struct3d (generic function with 1 method)

## Test Script

In [6]:
# function rand_out_struct3d(BX, BY, BZ, nRand, maxR)
N = 200;
@time BX = rand(N,N,N) * 10 - 5;
BY = rand(N,N,N) * 10 - 5;
BZ = rand(N,N,N) * 10 - 5;

  0.242395 seconds (10 allocations: 183.106 MiB, 64.05% gc time)


In [7]:
nRand = 15;
maxR = 30;
@time SF3d, NF = rand_out_struct3d(BX, BY, BZ, nRand, maxR)

  0.123498 seconds (9 allocations: 61.077 MiB, 89.40% gc time)
  0.000015 seconds
  0.713890 seconds
  0.838572 seconds (1.46 k allocations: 61.131 MiB, 13.17% gc time)


([NaN, 1762.8, 3173.62, 1792.02, 1489.15, 6686.77, 7014.45, 0.0, 3619.43, 8364.94  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1958.43], [15.0, 90.0, 180.0, 120.0, 90.0, 360.0, 360.0, 0.0, 180.0, 450.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 120.0])

In [ ]:
# @time x = sqrt.(1:3*maxR^2+1);
# plot(x,SF3d./NF)